## 決策樹實踐
- 說明: 決策樹是隨機森林以及梯度提升樹的基礎，也是知名演算法LightGBM、XGBoost的基礎(因為本質就是梯度提升樹)，其模型透過如同人類決策般不斷根據特徵以及閾值進行切分，最終將原始資料分類或者回歸(平均)。
- 演算法步驟:
    1. 選擇一個特徵
    2. 決定閾值
    3. 計算Gini(/Entropy)[分類問題]、MSE(/MAE)[回歸問題]
    4. 如果父節點的分數 < 子節點根據比例加權則代表可切分，並且記錄feature+threshold+y_hat
        如果 > 則回到步驟1.
    5. 當節點的樣本數、或者樹深度達到限制條件則停止，若不限制增長則會長到所有都是葉節點。
- 優化方向
    - 選擇最佳拆分特徵以及閾值 ---> 貪婪算法。
    
---
都以簡化實現，僅把主要邏輯實現為主，比如如何挑選特徵、如何找閾值。

In [229]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.datasets import fetch_california_housing

%matplotlib inline

In [231]:
## 測試資料

data = fetch_california_housing(as_frame=True)
X = data['data']
y = data['target']

In [5]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [6]:
y

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: MedHouseVal, Length: 20640, dtype: float64

In [23]:
## 拆分邏輯計算

# 1. 選擇一個特徵
feature_name = "HouseAge"
feature_name = np.random.choice(X.columns)
print(f"當前選擇特徵: {feature_name}")

# 2. 決定threshold
# 2.1 上下限從均勻選擇(可更新成其他選擇方式)

_max = X[feature_name].max()
_min = X[feature_name].min()
threshold = np.random.uniform(low=_min, high=_max, size=1)[0]
print(_min, _max, threshold)

# 3. 計算MSE
score_function = mean_absolute_error
current_sample_idx = X[feature_name].index
current_y_hat = y[current_sample_idx].mean()
current_y_hat = [current_y_hat] * len(current_sample_idx)

current_mse = score_function(y, current_y_hat)
print(f'當前節點(父)MSE: {current_mse}')

# 根據threshold 取得左節點、右節點
left_node = X[X[feature_name] < threshold]
right_node=  X[X[feature_name] >= threshold]

left_num_samples = len(left_node)
right_num_samples = len(right_node)


left_y = y[left_node.index]
right_y = y[right_node.index]

total_samples = len(left_y) + len(right_y)

left_y_hat = [left_y.mean()] * left_num_samples
right_y_hat = [right_y.mean()] *right_num_samples
left_mse = score_function(left_y, left_y_hat)
right_mse = score_function(right_y, right_y_hat)
print(f'左節點MSE: {left_mse}')
print(f'右節點MSE: {right_mse}')

# 計算切分分數
left_weight = left_num_samples / total_samples
right_weight = right_num_samples / total_samples
sub_sample_mse = left_mse * left_weight + right_mse * right_weight
print(f'左右加權MSE: {sub_sample_mse}')

# 判斷是否當前切分有比較好
is_more_pure = (current_mse > sub_sample_mse)
print(f"當前切分是否更pure: {is_more_pure}")

當前選擇特徵: Latitude
32.54 41.95 36.454363166585765
當前節點(父)MSE: 0.9117043994367803
左節點MSE: 0.8802840637029075
右節點MSE: 0.9546865582524363
左右加權MSE: 0.910874159087951
當前切分是否更pure: True


In [100]:
## 當訓練完時，有一個規則去判斷
class RuleNode():
    def __init__(self, feature_name, threshold, num_samples, mse, value):
        self.feature_name = feature_name
        self.threshold = threshold
        self.num_samples = num_samples
        self.mse = mse
        self.value = value
        self.left = None
        self.right = None
        self.has_left = False
        self.has_right = False
    
    def get_prediction(self):
        return self.value
    
    def __repr__(self):
        return f"當前Node:\n-------\nfeature_name: {self.feature_name}\nthreshold: {self.threshold}\nnum_samples: {self.num_samples}\nmse: {self.mse}\nvalue: {self.value}\nhas_left: {self.has_left}\nhas_right: {self.has_right}\n-------"
        
current_node = RuleNode(feature_name, threshold, total_samples, current_mse, np.mean(current_y_hat))
current_node
# current_node.value

當前Node:
-------
feature_name: Latitude
threshold: 34.53896866454806
num_samples: 20640
mse: 0.9117043994367803
value: 2.068558169089184
has_left: False
has_right: False
-------

In [236]:
## 實踐一個回歸的，分類比較常有人做

class DecisionTreeRegressorSimple():
    def __init__(self, max_depth=None, criterion='MSE', min_samples_split=2, random_state=222):
        np.random.seed(random_state)
        self.max_depth = max_depth
        self.criterion = criterion
        if self.criterion == 'MSE':
            self.score_function = mean_squared_error
        else:
            self.score_function = mean_absolute_error
        self.min_samples_split = min_samples_split
        self.records = {}
        self.count = 0
        self.max_retry = 10
    
    def _get_random_feature(self, X):
        features = X.columns
        return np.random.choice(features)
    
    def _get_best_split_feature(self, X):
        ## TODO: 需要仔細看一下
#         best_feature = None
#         current_best_score = None
#         for feature in X.columns:
#             if best_feature is None:
#                 best_feature = feature
#             threshold = self._get_feature_threshold(X, feature)
#             # 父節點計算
#             current_sample_idx = X[feature].index
#             current_y_hat = y[current_sample_idx].mean()
#             total_samples = len(X)
#             current_mse = self.score_function(y, [current_y_hat] * total_samples)
            
#             # 切出左節點、右節點
#             left_X = X[X[feature] < threshold]
#             right_X =  X[X[feature] >= threshold]
#             left_num_samples = len(left_X)
#             right_num_samples = len(right_X)
#             left_y = y[left_X.index]
#             right_y = y[right_X.index]
#             left_y_hat = [left_y.mean()] * left_num_samples
#             right_y_hat = [right_y.mean()] * right_num_samples
#             left_score = self.score_function(left_y, left_y_hat)
#             right_score = self.score_function(right_y, right_y_hat)
#             left_weight = left_num_samples / total_samples
#             right_weight = right_num_samples / total_samples
#             sub_sample_mse = left_score * left_weight + right_score * right_weight
#             if current_best_score is None:
                
            
        
        return
    
    def _get_feature_threshold(self, X, feature):
        _max = X[feature].max()
        _min = X[feature].min()
        threshold = np.random.uniform(low=_min, high=_max, size=1)[0]
        return threshold
    
    def _record_rules(self):
        """
        當每次切分成功後，需要紀錄切分的`特徵`、`score`、`samples`、`value(y_hat)`
        """
        pass
    
    def _split_data(self, X, y, depth=0, retry_count=0):
        ## TODO: 沒有設計好，要思考一下！
        if len(X) == 0:
            return None
        if depth == self.max_depth:
            print(f'深度太深，請檢查')
            return None
        
        # 選擇特徵、閾值
        feature = self._get_random_feature(X)
        threshold = self._get_feature_threshold(X, feature)
        
        # 父節點計算
        current_sample_idx = X[feature].index
        current_y_hat = y[current_sample_idx].mean()
        total_samples = len(X)
        current_mse = self.score_function(y, [current_y_hat] * total_samples)
        
        # 父節點建立
        current_node = RuleNode(feature, threshold, total_samples, current_mse, current_y_hat)
        if depth == 0:
            self.root = current_node
        
        # 切出左節點、右節點
        left_X = X[X[feature] < threshold]
        right_X =  X[X[feature] >= threshold]
        left_num_samples = len(left_X)
        right_num_samples = len(right_X)
        left_y = y[left_X.index]
        right_y = y[right_X.index]
        
        if (left_num_samples == 0) or (right_num_samples == 0):
            #print(f'終結節點的樣本數: {total_samples}')
            #print(f'確認終端節點: 左邊: {left_num_samples}, 右邊: {right_num_samples}')
            #print(f'此時threshold: {threshold}, max: {X[feature].max()}, min: {X[feature].min()}')
            
            # 因為少了一邊都不需要切了
            current_node.left = None
            current_node.right = None
            return current_node

        # 計算左右節點切分分數
        left_y_hat = [left_y.mean()] * left_num_samples
        right_y_hat = [right_y.mean()] * right_num_samples
        left_score = self.score_function(left_y, left_y_hat)
        right_score = self.score_function(right_y, right_y_hat)
        left_weight = left_num_samples / total_samples
        right_weight = right_num_samples / total_samples
        sub_sample_mse = left_score * left_weight + right_score * right_weight
        #print(current_mse, sub_sample_mse, left_weight, right_weight)

        # 判斷是否當前切分有比較好
        is_more_pure = (current_mse > sub_sample_mse)
        if is_more_pure:
            #print(f'Pure時，切分特徵: {feature}, threhold: {threshold}, 左節點: {left_num_samples}, 右節點: {right_num_samples}')
            #print(f"is_more_pure: {True}")
            current_node.left = self._split_data(left_X, left_y, depth+1)
            current_node.right = self._split_data(right_X, right_y, depth+1)
            if current_node.left:
                current_node.has_left = True
            if current_node.right:
                current_node.has_right = True
        else:
            if retry_count == self.max_retry:
                ## 避免一直找不到造成錯誤
                current_node.left = None
                current_node.right = None
                return current_node
            return self._split_data(X, y, depth, retry_count+1)
        return current_node
        
        
    def fit(self, X, y):
        self._split_data(X, y, 0)
    
    def predict(self, X, next_node=None, is_start=True):
        """ 透過Rule搭配boolean matricx快速判斷。 """
        if is_start:
            self.pred = X.copy()
            self.pred['label'] = 0
            current_node = self.root
        else:
            current_node = next_node
        feature = current_node.feature_name
        threshold = current_node.threshold
        value = current_node.value
        
        #print(current_node)
        
        if (not current_node.has_left) and (not current_node.has_right):
            #print(value)
            ## 到了終端節點
            idx = X.index
            self.pred.iloc[idx, -1] = value
            return self.pred.iloc[:, -1]
        
        if current_node.has_left:
            # 有左邊
            X_left = X[X[feature] < threshold]
            self.predict(X_left, current_node.left, False)
        if current_node.has_right:
            # 有右邊
            X_right = X[X[feature] >= threshold]
            self.predict(X_right, current_node.right, False)
        return self.pred.iloc[:, -1]

In [237]:
%%time

dt = DecisionTreeRegressorSimple()
dt.fit(X, y)

Wall time: 2min 43s


In [238]:
dt.predict(X)

0        4.526000
1        3.585000
2        3.521000
3        3.859667
4        3.422000
           ...   
20635    0.781000
20636    0.771000
20637    0.923000
20638    0.847000
20639    0.894000
Name: label, Length: 20640, dtype: float64

In [239]:
dt.pred['true_label'] = y

In [240]:
for i in range(dt.pred.shape[0]):
    print(f"第{i+1}筆資料, 真實標籤: {dt.pred.iloc[i, -1]}, 預測: {dt.pred.iloc[i, -2]}")

第1筆資料, 真實標籤: 4.526, 預測: 4.526
第2筆資料, 真實標籤: 3.585, 預測: 3.585
第3筆資料, 真實標籤: 3.521, 預測: 3.521
第4筆資料, 真實標籤: 3.413, 預測: 3.859666666666667
第5筆資料, 真實標籤: 3.422, 預測: 3.422
第6筆資料, 真實標籤: 2.697, 預測: 2.697
第7筆資料, 真實標籤: 2.992, 預測: 2.992
第8筆資料, 真實標籤: 2.414, 預測: 2.414
第9筆資料, 真實標籤: 2.267, 預測: 2.267
第10筆資料, 真實標籤: 2.611, 預測: 2.611
第11筆資料, 真實標籤: 2.815, 預測: 2.815
第12筆資料, 真實標籤: 2.418, 預測: 2.418
第13筆資料, 真實標籤: 2.135, 預測: 2.135
第14筆資料, 真實標籤: 1.913, 預測: 2.2936666666666667
第15筆資料, 真實標籤: 1.592, 預測: 1.592
第16筆資料, 真實標籤: 1.4, 預測: 1.4
第17筆資料, 真實標籤: 1.525, 預測: 1.525
第18筆資料, 真實標籤: 1.555, 預測: 2.5244400000000002
第19筆資料, 真實標籤: 1.587, 預測: 1.587
第20筆資料, 真實標籤: 1.629, 預測: 1.629
第21筆資料, 真實標籤: 1.475, 預測: 1.475
第22筆資料, 真實標籤: 1.598, 預測: 1.598
第23筆資料, 真實標籤: 1.139, 預測: 1.139
第24筆資料, 真實標籤: 0.997, 預測: 0.997
第25筆資料, 真實標籤: 1.326, 預測: 1.326
第26筆資料, 真實標籤: 1.075, 預測: 1.075
第27筆資料, 真實標籤: 0.938, 預測: 0.938
第28筆資料, 真實標籤: 1.055, 預測: 1.055
第29筆資料, 真實標籤: 1.089, 預測: 1.089
第30筆資料, 真實標籤: 1.32, 預測: 1.32
第31筆資料, 真實標籤: 1.223, 預測: 1.223
第32筆資料, 真實標籤: 1.

In [241]:
## 查看MSE、MAE
mse = round(mean_squared_error(dt.pred.iloc[:, -1], dt.pred.iloc[:, -2]), 4)
mae = round(mean_absolute_error(dt.pred.iloc[:, -1], dt.pred.iloc[:, -2]), 4)
mape = round(mean_absolute_percentage_error(dt.pred.iloc[:, -1], dt.pred.iloc[:, -2]), 4)

print(f"MAPE: {mape}, MAE: {mae}, MSE: {mse}")

MAPE: 0.0158, MAE: 0.028, MSE: 0.0241


In [242]:
%%time
## 使用sklearn DT

dt_sklearn = DecisionTreeRegressor().fit(X, y)

Wall time: 394 ms


In [243]:
## 查看MSE、MAE
y_pred = dt_sklearn.predict(X)

mse = round(mean_squared_error(dt.pred.iloc[:, -1], y_pred), 4)
mae = round(mean_absolute_error(dt.pred.iloc[:, -1], y_pred), 4)
mape = round(mean_absolute_percentage_error(dt.pred.iloc[:, -1], y_pred), 4)

print(f"MAPE: {mape}, MAE: {mae}, MSE: {mse}")

MAPE: 0.0, MAE: 0.0, MSE: 0.0
